In [1]:
from core.board import Board
from analysis.msm_builder import get_msm_graph
from solver.solver import solve

In [2]:
r = 16
c = 30
n = 99
start = (0,0)
seed = None        
preset = False
profile = False
B_t = Board.random_complete(r,c,n,exclude=[start],seed=seed)
B_k = Board(r,c,n)
if preset:
    B_k.set_clear(0,0,1)
    B_k.set_clear(1,0,1)
    B_k.set_clear(2,0,1)
    B_k.set_clear(2,1,3)
    B_k.set_clear(0,2,2)
if preset: print(B_k)
elif seed is not None: print(B_t)

In [3]:
from analysis.msm_analysis import find_solutions, seperate_connected_msm_components
if preset:
    B_k = Board(r,c,n)
    B_o, msm, c, f = get_msm_graph(B_k, order=2, verbose=0)
    print(B_o, msm, c, f)
    msms = seperate_connected_msm_components(msm)
    print(find_solutions(msms[0], verbose=0))

In [4]:
import cProfile
if not preset and seed is not None:
    B_k = Board(r,c,n)
    if profile:
        cProfile.run('print(solve(B_k, B_t, first_move=start, verbose=0))')
    else:
        print(solve(B_k, B_t, first_move=start, verbose=0))

In [5]:
import random
if seed is None and not preset:
    start = (0,0)
    max_iterations = 1000
    max_wins = 200
    wins = 0
    win_time = 0
    win_guesses = 0
    win_risk = 0
    ng_games = 0
    i = 0
    for i in range(max_iterations):
        seed = random.randint(0, 2**32)
        B_t = Board.random_complete(r,c,n,exclude=[start], seed=seed)
        B_k = Board(r,c,n)
        print(seed, end=' ')
        win, stats = solve(B_k, B_t, first_move=start, verbose=0)
        if win: 
            wins += 1
            win_time += stats.time
            win_guesses += stats.guesses
            win_risk += stats.risk
            if stats.guesses == 0: ng_games += 1
        print(win, stats)
        if wins >= max_wins: break
    seed = None
    i += 1
    print(f'Wins: {wins/i}, Avg_Time: {win_time/wins}, Avg_guess: {win_guesses/wins}, Avg_risk:{win_risk/wins}, NG: {ng_games}')

618380856 False [Time: 0.377, Efficiency: 18%, Risk: 9.48%, Guesses: 1, Uncaught Logic: 0]
766293550 False [Time: 2.073, Efficiency: 3%, Risk: 64.73%, Guesses: 7, Uncaught Logic: 1]
2902425673 True [Time: 1.926, Efficiency: 6%, Risk: 43.37%, Guesses: 4, Uncaught Logic: 0]
4254194137 True [Time: 13.892, Efficiency: 5%, Risk: 95.17%, Guesses: 17, Uncaught Logic: 0]
101593656 False [Time: 0.019, Efficiency: 1400%, Risk: 20.59%, Guesses: 1, Uncaught Logic: 0]
3464306891 True [Time: 5.177, Efficiency: 6%, Risk: 47.97%, Guesses: 9, Uncaught Logic: 0]
180873676 False [Time: 0.837, Efficiency: 475%, Risk: 44.48%, Guesses: 3, Uncaught Logic: 0]
275723139 False [Time: 1.060, Efficiency: 6%, Risk: 60.29%, Guesses: 2, Uncaught Logic: 0]
577808299 False [Time: 2.644, Efficiency: 4%, Risk: 50.00%, Guesses: 1, Uncaught Logic: 0]
567064945 False [Time: 1.819, Efficiency: 9%, Risk: 34.72%, Guesses: 3, Uncaught Logic: 0]
3413770316 True [Time: 3.110, Efficiency: 6%, Risk: 43.56%, Guesses: 4, Uncaught Lo